In [ ]:

import plotly.express as px
import pandas as pd

import sys; sys.path.append('../..')
from src.reporting.trades import read_trades

from src.outputs import pathing

input_path = pathing.get_paths()['data']["dir"] + '/options_trades.jsonl'
trades = list(read_trades(input_path))

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import datetime
import typing
import collections
import itertools
from src.backtest import brackets_historical

tps = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
sls = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
points = []
for tp, sl in itertools.product(
    tps,
    sls,
):
    # if sl > tp:
    #     continue

    bracket_set: list[brackets_historical.Bracket] = [{'take_profit_percentage': tp, 'stop_loss_percentage': sl, 'until': datetime.time(15, 59)}]
    bracketed_trades = [brackets_historical.get_bracketed_trade(t, bracket_set) for t in trades]
    profit = sum(t['profit_loss'] for t in bracketed_trades)
    print(tp, sl, collections.Counter(t['is_win'] for t in bracketed_trades), profit)

    points.append({
        'tp': tp,
        'sl': sl,
        'profit': profit,
    })


In [58]:

fig = px.density_heatmap(pd.DataFrame(points), x="sl", y="tp", z="profit", nbinsx=len(sls), nbinsy=len(tps))
fig.show()

In [ ]:
for sl, tp in itertools.product(sls, tps):
    bracket_set= typing.cast(
        list[brackets_historical.Bracket],
        [{'take_profit_percentage': tp, 'stop_loss_percentage': sl, 'until': datetime.time(15, 59)}]
    )
    bracketed_trades = [brackets_historical.get_bracketed_trade(t, bracket_set) for t in trades]
    fig = px.histogram(
        pd.DataFrame(bracketed_trades),
        x='roi',
        title=f'Bracketed trades sl={sl} tp={tp}',
        marginal='rug',
        range_x=[-1, 1],
    )
    fig.show()
    print("^^^", sl, tp, collections.Counter(t['is_win'] for t in bracketed_trades),
          sum(t['profit_loss'] for t in bracketed_trades))